# SiameseBert Semantic Sentence Similiarity

In [1]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [2]:
# # A corpus is a list with documents split by sentences.
# sentences = ['clock', 'time', 'how many hours', 'minutes',"date",'date', 'calendar',"schedule",'timetable','plan','agenda','task list', 'things to do', 'todo list', 'work','music','play music', 'rythm','melody']
# # Each sentence is encoded as a 1-D vector with 78 columns
# sentence_embeddings = model.encode(sentences)

# print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

# print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])
import scipy
import numpy as np
# feature_db = { "insane":['Absence of sanity', 'Lack of saneness'],
#                 "rocket" : ['Launch','Build']}
# feature_db = { "time":['clock', 'time', 'how many hours', 'minutes',],
# 		"date":['date', 'calendar',],
# 		"schedule":['timetable','plan','agenda','task list', 'things to do', 'todo list','work'],
# 		"music": ['music','play music', 'rythm','melody'],}
feature_db = {
		"time":['time','clock','watch','deadline','how much time is the clock showing','what time zone are you following','what is the time right now',"what's the time","What time is it","tell me the time",'tell me time in london','tell me time in usa','tell me time in sydney right now','what is time in australia right now','right now','at this moment'],

		"date":["date","occasion","day","month","year",'sunday','monday','tuesday','friday','wednesday',"todays's date","what day is it today","what is the occasion today","is there something special today","why is there a public holiday tomorrow","working day","weekends"],

		"location":["location","where am i","whats my location","where am i right now exact address","where am i ip","where am i address","where am i in the world","olivia where am i","where am i right now app","what city am i in","my address","which area am i in",'tell me where am i'],

		"weather":["how is weather today","how is the weather in delhi","is it raining today","are clouds today","weather","how much is teamperature","is humidity to high","humidity","temperature","winds","how fast are the winds","sunny weather","cloud weather","humid atmosphere","atomspheric pressure"],

		"alarm reminder":["alarm","reminder","remind me to do","alert for when this happens","remind me after 1 hour to do this","set a reminder for 10 am tomorrow","set an alarm for today 11 am"],

		"schedule" :["schedule","plan","agenda","meeting","deadline","plan of action","plan of work","what the schedule for tomorrow","whats the schedule for now","set a meeting with abc at 2pm on thursday this week"],

		"music":["music","song","bgm","playlist","play","play music","play kaho na kaho", "play Ooo.ohh..ooooo", "play pai papa pari parari ba pari","I want to listen sonu's best songs"],

		"find information": ["find information","find me info about this","get me details on that","find profile of nisha pankaj","find every possible thing on elon musk","find details about google company","where did abhinav mishra study","what is the total number of cases for today","who is brother of kiara","how many people died in blm"],

		"message":["message","chat","whatsapp","instagram dm","message bob and tell him i'll be late","inform mom to get dinner ready","inform them to start","inform neha that i am coming",'chat with ajay and find what is he doing'],
		
		"email":["email","mail","email conversation","gmail","microsoft outlook","email bob and tell him i'll be late","mail manish to get ppt ready by tomorrow","email paul and inform him to start","email abc and do","type a mail to bob","send a mail to neha and inform about the updates","email bob and inform him i'll be late"],

		"call":['phone call',"call mom and inform her i'll be late","call puneet and tell him to sell it today","call abc",'handle my all calls and put the phone to bussy mode','address abc with details when you receive call from him','tell john to move when he calls','call','phone puneet and tell him to sell it today','phone abc','phone rebbecaa to inform her about'],
		
		"features":['feature list','what can you do','what are your utilities','tell me about your feature menu','what could you do for me','how good you are','who created you'],

		"translation":['translate','language translate','translate to german','translate to french','change to russion','translate this to english language','language translate'],
#		"calculator":[],
#		"trending news":[''],	
	}


feature_db_encoded = {}
for key,v in feature_db.items():
    feature_db_encoded[key]=model.encode(v)


In [6]:
n=4
query="schedule a meeting with xyz and set reminder in case i forget."
def n_gram_gen(n):
    n_gram=[]
    words=query.split(" ")
    for i in range(len(words)-n+1):
        n_gram.append(' '.join(words[i:i+n]))
    return n_gram
n_gram=n_gram_gen(n)
n_gram

['schedule a meeting with',
 'a meeting with xyz',
 'meeting with xyz and',
 'with xyz and set',
 'xyz and set reminder',
 'and set reminder in',
 'set reminder in case',
 'reminder in case i',
 'in case i forget.']

In [7]:
#@title Sematic Search Form

# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
query_embedding = model.encode(n_gram)
results=[]
key_res=[]
# distances = scipy.spatial.distance.cdist(query_embedding, feature_db_encoded['time'], "cosine")
# print(distances)
# print (np.transpose(distances))
for key,value in feature_db_encoded.items():
    key_enc = model.encode([key])
    key_dist = scipy.spatial.distance.cdist(query_embedding, key_enc, "cosine")
    distances = scipy.spatial.distance.cdist(query_embedding, value, "correlation")
    distances = np.transpose(distances)
    distances = [sorted(x) for x in distances]
    distances = [np.mean(x[:int(len(n_gram)/2)]) for x in distances]
    results.append(np.mean(distances)*np.mean(key_dist))
    key_res.append(np.mean(key_dist))
    # results.append(np.mean(distances))
res=zip(list(feature_db.keys()),key_res)
results = zip(list(feature_db.keys()), results)
results = sorted(results, key=lambda x: x[1])
results
# results1=results
# results1=[ [t[0],t[1]/results[-1][1]] for t in results1]
# print (results1)

[('alarm reminder', 0.22915514861985556),
 ('schedule', 0.23382565411938083),
 ('location', 0.2702584080289576),
 ('message', 0.2746785922971736),
 ('time', 0.28920664010047986),
 ('find information', 0.30805807016384756),
 ('call', 0.3185568514351925),
 ('features', 0.3237562089396498),
 ('date', 0.3282701017431382),
 ('music', 0.4455524035879113),
 ('translation', 0.44661855267522466),
 ('email', 0.46677234532358897),
 ('weather', 0.5076121011773285)]

In [8]:
res=sorted(res,key=lambda x:x[1])
res

[('location', 0.44735160251616624),
 ('find information', 0.48718177154872233),
 ('schedule', 0.4971556450784258),
 ('features', 0.502810785088071),
 ('time', 0.5128494136628485),
 ('call', 0.5152873730027063),
 ('message', 0.5386769134681312),
 ('date', 0.551721428204862),
 ('alarm reminder', 0.562697540625121),
 ('translation', 0.6465950535165081),
 ('weather', 0.6826417086204569),
 ('music', 0.7142249193649687),
 ('email', 0.729240557854202)]